In [34]:
import pandas as pd
import os
import glob
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter    


In [35]:
def data_name(path):
        '''
        Function to read the names of all the files in the folder.
        File name should be in a format: "Liquid_Substrate_Repeat.txt"
        '''
        files = glob.glob(os.path.join(path, '*.txt'))
        data_name = []
        for i in range(len(files)):
            x = str(files[i]).split('/')
            current_name = x[-1].split('.')[0]
            data_name.append(current_name)
        return data_name

In [36]:
def find_ACA(data, max_difference = 1.0, min_size = 5, max_size = 50):
    '''
    Function to find the maximum part of advancing contact angle plateau. 
    max_difference is the maximum change in contact angle allowed in the plateau, default is 1.0.
    min_size is the minimum size of the plateau, default is 5.
    max_size is the maximum size of the plateau, default is 50.
    '''
    data_advance = data[data['Status'] == 'Advancing']
    ACA_plateau = pd.DataFrame(columns=data_advance.columns)
    chunk_size = max_size
    while chunk_size >= min_size:
        for index in range(len(data_advance) - chunk_size):
            chunk = data_advance[index : index + chunk_size]
            d = chunk["Data_Angle"].max() - chunk["Data_Angle"].min()
            if d < max_difference:
                ACA_plateau = ACA_plateau.append(chunk)
        if ACA_plateau.empty == False:
            break
        chunk_size = chunk_size - 1
    return ACA_plateau


def find_RCA(data, max_dAngle = 1.0, min_dBase = 0.1, min_size = 3, max_size = 50):
    '''
    Function to find the maximum part of receding contact angle plateau.
    max_dAngle is the maximum change in contact angle allowed in the plateau, default is 1.0.
    min_dBase is the minimum change in base length allowed in the plateau, default is 0.1.
    min_size is the minimum size of the plateau, default is 3.
    max_size is the maximum size of the plateau, default is 50.
    '''
    data_receding = data[data['Status'] == 'Receding']
    RCA_plateau = pd.DataFrame(columns=data_receding.columns)
    chunk_size = max_size
    while chunk_size >= min_size:
        for index in range(len(data_receding) - chunk_size):
            chunk = data_receding[index : index + chunk_size]
            da = chunk["Smoothed_Angle"].max() - chunk["Smoothed_Angle"].min()
            db = chunk["Smoothed_Base"].max() - chunk["Smoothed_Base"].min()
            if (da < max_dAngle) & (db > min_dBase):
                RCA_plateau = RCA_plateau.append(chunk)
        if RCA_plateau.empty == False:
            break
        chunk_size = chunk_size - 1
    return RCA_plateau


In [42]:
def sum_CA(path, print_origdata = False, print_CA = False):
    '''
    Function to summarize the data and calculate the advancing and receding contact angles.

    print_origdata is a boolean to print the original data, default is False.
    print_CA is a boolean to print the contact angle data, default is False.

    Output:
    Output[0]: Tables of average values of advancing and receding contact angles and their standard deviations.
    Output[1] (if print_origdata == True): Dictionary of original data.
    Output[2] (if print_CA == True): Dictionary of contact angle data.
    '''
    files = glob.glob(os.path.join(path, '*.txt'))
    
    data_name = []
    for i in range(len(files)):
        x = str(files[i]).split('/')
        current_name = x[-1].split('.')[0]
        data_name.append(current_name)
    
    data_list = dict()
    CA_list = dict()
    sum_table = pd.DataFrame(columns=['ACA [˚]', 'ACA std', 'RCA [˚]', 'RCA std', 'Replica'])
    test_number = len(files)
    data_ACA = np.zeros(test_number)
    ACA_std = np.zeros(test_number)
    data_RCA = np.zeros(test_number)
    RCA_std = np.zeros(test_number)
    liquid = [" "] * test_number
    replica = np.zeros(test_number)
    substrate = [" "] * test_number

    for i in range(len(files)):
        # import data
        current_df = pd.read_fwf(files[i],header =None)
        # mix first two rows for headers
        current_df.columns = (current_df.iloc[0] + '_' + current_df.iloc[1])
        current_df = current_df.iloc[2:].reset_index(drop=True)
        
        # extract wanted columns time, angle and base
        current_data = current_df[['Graph_Secs','Data_Angle', 'Data_Base']]
        current_df = current_df.dropna(axis=0)
        current_data = current_data.astype(float)

        # get smoothed data, remove noise
        num1 = current_data.shape[0]//3
        if num1%2 == 0:
            current_data['Smoothed_Angle'] = savgol_filter(current_data['Data_Angle'], window_length=(num1+1), polyorder=3)
        else:
            current_data['Smoothed_Angle'] = savgol_filter(current_data['Data_Angle'], window_length=num1, polyorder=3)
        
        num2 = current_data.shape[0]//2
        if num2%2 == 0:
            current_data['Smoothed_Base'] = savgol_filter(current_data['Data_Base'], window_length=(num2+1), polyorder=3)
        else:
            current_data['Smoothed_Base'] = savgol_filter(current_data['Data_Base'], window_length=num2, polyorder=3)

        # change nan in gradient to 0
        base_gradient = np.nan_to_num(np.gradient(current_data.Smoothed_Base))
        current_data['Base_gradient'] = base_gradient.tolist()
        base_gradient_max = np.amax(base_gradient)

        # current_data = current_data[(current_data['Base_gradient'] < 0)|(current_data['Base_gradient'] >= 0.25*base_gradient_max)]
        current_data["Status"] = np.where(current_data.Base_gradient >= 0.25*base_gradient_max, 'Advancing', 'Receding')
        ind = current_data.loc[current_data["Status"] == 'Advancing'].index[0]
        current_data.drop(current_data.index[:ind], inplace=True)
        
        current_CA = current_data.drop(current_data[(current_data['Base_gradient'] >= 0) & (current_data['Status'] == 'Receding')].index, inplace=False)
        
        current_ACA = find_ACA(current_CA)
        current_RCA = find_RCA(current_CA)
        current_result = current_ACA.append(current_RCA)

        current_result['Substrate'] = str(data_name[i]).split('_')[1]
        current_result['liquid'] = str(data_name[i]).split('_')[0]
        current_result['Replica'] = int(str(data_name[i]).split('_')[2])

        data_ACA[i] = np.mean(current_ACA['Data_Angle'])
        ACA_std[i] = np.std(current_ACA['Data_Angle'])

        if current_RCA.empty == True:
            data_RCA[i] = 0
            RCA_std[i] = 0
        else:
            data_RCA[i] = np.mean(current_RCA['Data_Angle'])
            RCA_std[i] = np.std(current_RCA['Data_Angle'])

        liquid[i] = str(data_name[i]).split('_')[0]
        substrate[i] = str(data_name[i]).split('_')[1]
        replica[i] = int(str(data_name[i]).split('_')[2])


        data_list[data_name[i]] = current_data
        CA_list[data_name[i]] = current_result


    sum_table['ACA [˚]'] = data_ACA.tolist()
    sum_table['ACA std'] = ACA_std.tolist()
    sum_table['RCA [˚]'] = data_RCA.tolist()
    sum_table['RCA std'] = RCA_std.tolist()
    sum_table['Replica'] = replica.tolist()
    substrate_sum = pd.Series(substrate)
    sum_table.insert(loc=0, column='Substrate', value=substrate_sum)
    liquid_sum = pd.Series(liquid)
    sum_table.insert(loc=0, column='Liquid', value=liquid_sum)
    
    if print_origdata == False & print_CA == False:
        return sum_table
    elif print_origdata == True & print_CA == False:
        return sum_table, data_list
    else:
        return sum_table, data_list, CA_list

In [47]:
tables = sum_CA(path = 'path', print_origdata = True, print_CA = True)

/var/folders/t8/s3wbxd3j28sbrflrxg8_rz7x934gjg/T/ipykernel_58912/4018653903.py:103: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  substrate_sum = pd.Series(substrate)
/var/folders/t8/s3wbxd3j28sbrflrxg8_rz7x934gjg/T/ipykernel_58912/4018653903.py:105: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  liquid_sum = pd.Series(liquid)


In [ ]:
sum_table = tables[0]
data_list = tables[1]
CA_list = tables[2]

In [ ]:
# to plot the data

sns.lineplot(y =data_list[str(test_name)].Data_Angle, x = data_list[str(test_name)].Graph_Secs, color="g", label = "Angle", hue = data_list[str(test_name)].Status).set(title= str(test_name))
ax2 = plt.twinx()
sns.lineplot(y = data_list[str(test_name)].Smoothed_Base ,x = data_list[str(test_name)].Graph_Secs, color="b", ax=ax2, label = "Base")
ax2.legend(loc ="upper right", bbox_to_anchor=(1, 0.9))
plt.show()